# Import data

In [1]:
import kagglehub
path = kagglehub.dataset_download("grassknoted/asl-alphabet")
print("Path to dataset files:", path)

100%|██████████| 1.03G/1.03G [01:42<00:00, 10.8MB/s]

Extracting files...


Path to dataset files: C:\Users\ADMIN\.cache\kagglehub\datasets\grassknoted\asl-alphabet\versions\1


# Import lib

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Chuan hoa du lieu

In [4]:
IMG_SIZE = 64
BATCH_SIZE = 32

In [6]:
datagen = ImageDataGenerator(
    rescale=1./255, # chuyen doi gtri [0,255] ve [0,1]
    validation_split=0.2 # Chia du lieu thanh 80% train va 20% validation
)

In [10]:
train_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode="categorical",
    subset="training" # chi dinh lay tap train
)

val_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode="categorical",
    subset="validation" # chi dinh lay tap validation
)

Found 69623 images belonging to 2 classes.
Found 17405 images belonging to 2 classes.


# Khai bao mo hinh - fit

In [27]:
model = keras.Sequential
(
    [
        keras.layers.Conv2D(32,(3,3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        keras.layers.MaxPooling2D((2,2)),
        
        keras.layers.Conv2D(64,(3,3), activation="relu"),
        keras.layers.MaxPooling2D((2,2)),
        
        keras.layers.Conv2D(128,(3,3), activation="relu"),
        keras.layers.MaxPooling2D((2,2)),
        
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation="relu"), # fully connected layer
        keras.layers.Dropout(0.5), # dropout de tranh overfitting
        keras.layers.Dense(29,activation="softmax") # 29 classes trong asl
    ]
)

[<Conv2D name=conv2d_18, built=False>,
 <MaxPooling2D name=max_pooling2d_18, built=True>,
 <Conv2D name=conv2d_19, built=False>,
 <MaxPooling2D name=max_pooling2d_19, built=True>,
 <Conv2D name=conv2d_20, built=False>,
 <MaxPooling2D name=max_pooling2d_20, built=True>,
 <Flatten name=flatten_6, built=False>,
 <Dense name=dense_12, built=False>,
 <Dropout name=dropout_6, built=True>,
 <Dense name=dense_13, built=False>]

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# huan luyen
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

model.summary()

Epoch 1/10
2005/2176 ━━━━━━━━━━━━━━━━━━━━ 2:58 1s/step - accuracy: 0.9984 - loss: 0.0159

# Save model

In [ ]:
model.save("asl_alphabet_model.h5") # .h5 dung de load lai va du doan cho sau nay
print("Saved model")

# Danh gia mo hinh

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f"Accuracy: {accuracy:.4f}")
print(f"Loss: {loss:.4f}")

# Du doan

In [ ]:
# load mo hinh
loaded_model = keras.model.load_model("asl_alphabet_model.h5")
print("Loaded model from disk")

In [ ]:
# chuan bi label {chu cai}
class_names = list(train_data.class_indices.keys())
print(class_names)

In [ ]:
# load 1 hinh de xu ly
img_path = path + "/test.jpg" # duong dan den hinh can du doan
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.image_to_array(img)
img_array = img_array / 255.0 # scale anh
img_array = np.expand_dims(img_array, axis=0) # them kick thuoc

In [ ]:
# du doan
prediction = loaded_model.predict(img_array)
predicted_index = np.argmax(prediction)
predicted_label = class_names[predicted_index]

# print % cac lop du doan
print(prediction)
print(f"Predicted label: {predicted_label}")